In [1]:
import pandas as pd
import geopandas as gpd
import pandas as pd, geopandas as gpd
from shapely.geometry import Point 

In [2]:
csv_path     = "../../data/processed/scoring_indicators/housing_needs/merged_housing_data.csv"     
shp_path     = "../../data/raw/shapefiles/tl_2024_13_tract/tl_2024_13_tract.shp"
out_geojson  = "../../data/map/housing_need_characteristics/merged_housing_data.geojson"


In [36]:
csv_path     = "../../data/raw/scoring_indicators/food_access_research_atlas.csv"     
shp_path     = "../../data/raw/shapefiles/tl_2024_13_tract/tl_2024_13_tract.shp"
out_geojson  = "../../data/processed/scoring_indicators/food_access_research_atlas.geojson"


In [3]:
df = pd.read_csv(csv_path).rename(columns={"2020 Census Tract": "GEOID"})

# 👉 Convert tract column to an 11‑digit string (keeps leading zeros)
df["GEOID"] = (
    df["GEOID"]
      .astype(str)            # numeric → string
      .str.replace(r"\.0$", "", regex=True)  # drop any trailing “.0”
      .str.zfill(11)          # pad to 11 digits
)

# Read tract polygons
gdf = gpd.read_file(shp_path)[["GEOID", "geometry"]]

# Ensure polygon key is also a string
gdf["GEOID"] = gdf["GEOID"].astype(str).str.zfill(11)

# Merge
merged = gdf.merge(df, on="GEOID", how="inner")

In [4]:
merged.to_file(out_geojson, driver="GeoJSON")

In [2]:
csv_path_lat_lon = "../../data/map/application_list.csv"     
output_lat_lon = "../../data/map/application_list_2022_2023_2024_v2.geojson"  

In [7]:
csv_path_lat_lon = "../../data/processed/scoring_indicators/desirable_activities_google_places_v2.csv"     
output_lat_lon = "../../data/processed/scoring_indicators/desirable_activities_google_places_v3.csv"  

In [4]:
# 1. Read the 2024 GA census‑tract shapefile
ga_tracts = gpd.read_file("../../data/raw/shapefiles/tl_2024_13_tract/tl_2024_13_tract.shp").to_crs("EPSG:4326")

# 2. Convert your CSV to GeoDataFrame
df  = pd.read_csv(csv_path_lat_lon)

In [9]:
df.shape

(123769, 9)

In [5]:
gdf = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df.lon, df.lat),
        crs="EPSG:4326"
)

# 3. Keep only points that fall *within* a Georgia tract
gdf_ga = gpd.sjoin(gdf, ga_tracts[["GEOID", "geometry"]],
                   predicate="within", how="inner").drop(columns="index_right")

# # 4. Now write or analyse as needed
# gdf_ga.to_file(output_lat_lon, driver="GeoJSON")

In [11]:
gdf_ga.shape

(87463, 11)

In [13]:
gdf_ga.to_csv(output_lat_lon, index=False)


In [6]:
# 4. Now write or analyse as needed
gdf_ga.to_file(output_lat_lon, driver="GeoJSON")

In [ ]:
csv_path_lat_lon = "../../data/processed/scoring_indicators/undesirable_hsi_tri_cdr_rcra_google_places.csv"     
output_lat_lon = "../../data/processed/scoring_indicators/undesirable_hsi_tri_cdr_rcra_google_places.geojson"  

df  = pd.read_csv(csv_path_lat_lon)             
gdf = gpd.GeoDataFrame(df,
        geometry=gpd.points_from_xy(df.site_longitude, df.site_latitude),  # lon, lat order
        crs="EPSG:4326")                    # WGS‑84, the standard for GeoJSON

gdf.to_file(output_lat_lon, driver="GeoJSON")